# Export URDFs

Scans the `SOURCE_FOLDER` for `.obj` files, and for each mesh exports a URDF file and associated visual and collision shapes for simulation with pybullet to the `TARGET_FOLDER`.

In [ ]:
from __future__ import print_function, division

In [ ]:
import os
os.chdir('..')

In [ ]:
from pathlib2 import Path
import shutil
import tarfile

import h5py
import numpy as np
import pybullet as p
import requests
import trimesh

In [ ]:
SOURCE_FOLDER = Path("data/meshes/3dnet")
TARGET_FOLDER = Path("data/urdfs/3dnet")

SCALE_TO_FIT = True
MAX_OPENING_WIDTH = 0.08

In [ ]:
TARGET_FOLDER.mkdir(exist_ok=True)

In [ ]:
for mesh_path in SOURCE_FOLDER.iterdir():
    if not mesh_path.suffix == ".obj":
        continue

    name = mesh_path.stem
    urdf_path = TARGET_FOLDER / (name + ".urdf")
    visual_mesh_path = TARGET_FOLDER / (name + "_visual.obj")
    collision_mesh_path = TARGET_FOLDER / (name + "_collision.obj")

    print("Processing", name)
        
    # load the mesh
    mesh_in = trimesh.load(mesh_path)

    # check mesh
    if not mesh_in.is_watertight:
        print("Not watertight, skipping")
        continue
    
    # estimate physical properties
    density = 700
    mass = mesh_in.volume * density

    # export visual mesh
    mesh_visual = mesh_in.copy()
    mesh_visual.vertices -= mesh_visual.center_mass
    if SCALE_TO_FIT:
        extent = np.sort(mesh_visual.bounding_box.extents)[1]
        scale = min(1.0, (MAX_OPENING_WIDTH - 0.01) / extent)
        mesh_visual.vertices *= scale
    mesh_visual.export(visual_mesh_path)

    # export collision mesh
    p.vhacd(str(visual_mesh_path), str(collision_mesh_path), "log.txt")

    # export urdf
    urdf = (
        '<?xml version="1.0"?>\n'
        '<robot name="{}">\n'
        '  <link name="base_link">\n'
        "    <contact>\n"
        '      <lateral_friction value="1.0"/>\n'
        '      <rolling_friction value="0.0"/>\n'
        '      <contact_cfm value="0.0"/>\n'
        '      <contact_erp value="1.0"/>\n'
        "    </contact>\n"
        "    <inertial>\n"
        '      <mass value="{}"/>\n'
        '      <inertia ixx="1" ixy="0" ixz="0" iyy="1" iyz="0" izz="1"/>\n'
        "    </inertial>\n"
        "    <visual>\n"
        "      <geometry>\n"
        '        <mesh filename="{}"/>\n'
        "      </geometry>\n"
        "    </visual>\n"
        "    <collision>\n"
        "      <geometry>\n"
        '        <mesh filename="{}"/>\n'
        "      </geometry>\n"
        "    </collision>\n"
        "  </link>\n"
        "</robot>\n"
        ""
    ).format(name, mass, visual_mesh_path.name, collision_mesh_path.name)
    with urdf_path.open("wb") as f:
        f.write(urdf)

print("Done")

## BigBIRD

In [ ]:
CWD = Path.home() / "Downloads"

In [ ]:
objects = [
    "3m_high_tack_spray_adhesive",
    "advil_liqui_gels",
    "aunt_jemima_original_syrup",
    "bai5_sumatra_dragonfruit",
    "band_aid_clear_strips",
    "band_aid_sheer_strips",
    "blue_clover_baby_toy",
    "bumblebee_albacore",
    "campbells_chicken_noodle_soup",
    "campbells_soup_at_hand_creamy_tomato",
    "canon_ack_e10_box",
    "cheez_it_white_cheddar",
    "chewy_dipps_chocolate_chip",
    "chewy_dipps_peanut_butter",
    "cholula_chipotle_hot_sauce",
    "cinnamon_toast_crunch",
    "clif_crunch_chocolate_chip",
    "coca_cola_glass_bottle",
    "coffee_mate_french_vanilla",
    "colgate_cool_mint",
    "crayola_24_crayons",
    "crest_complete_minty_fresh",
    "crystal_hot_sauce",
    "cup_noodles_chicken",
    "detergent",
    "dove_beauty_cream_bar",
    "eating_right_for_healthy_living_apple",
    "expo_marker_red",
    "fruit_by_the_foot",
    "haagen_dazs_butter_pecan",
    "haagen_dazs_cookie_dough",
    "hersheys_bar",
    "hersheys_cocoa",
    "honey_bunches_of_oats_honey_roasted",
    "hunts_paste",
    "ikea_table_leg_blue",
    "krylon_crystal_clear",
    "krylon_short_cuts",
    "mom_to_mom_butternut_squash_pear",
    "motts_original_assorted_fruit",
    "nice_honey_roasted_almonds",
    "nutrigrain_apple_cinnamon",
    "palmolive_green",
    "pepto_bismol",
    "pop_secret_butter",
    "pop_secret_light_butter",
    "pop_tarts_strawberry",
    "pringles_bbq",
    "progresso_new_england_clam_chowder",
    "quaker_big_chewy_chocolate_chip",
    "red_bull",
    "red_cup",
    "softsoap_gold",
    "south_beach_good_to_go_dark_chocolate",
    "spam",
    "spongebob_squarepants_fruit_snaks",
    "suave_sweet_guava_nectar_body_wash",
    "sunkist_fruit_snacks_mixed_fruit",
    "v8_fusion_peach_mango",
    "vo5_extra_body_volumizing_shampoo",
    "white_rain_sensations_apple_blossom_hydrating_body_wash",
    "windex",
    "zilla_night_black_heat",
]

In [ ]:
for name in objects:
    print("Processing", name)
    
    # download
    url = "http://rll.berkeley.edu/bigbird/aliases/dca39bef51/export/" + name + "/processed.tgz"
    archive = CWD / "processed.tgz"
    r = requests.get(url, allow_redirects=True)
    archive.open("wb").write(r.content)
    
    # extract
    tar = tarfile.open(str(archive))
    tar.extractall(str(CWD))
    tar.close()
    archive.unlink()
    
    # copy
    source_path = CWD / name / "meshes" / "poisson.ply"
    target_path = CWD / "bigbird" / (name + ".ply")
    shutil.copy(str(source_path), str(target_path))

## Kappler Grasp Database

The grasp database can be downloaded [here](http://grasp-database.dkappler.de/).

In [ ]:
CWD = Path.home() / "Downloads"

In [ ]:
f = h5py.File(str(CWD / "grasp-database-all-v01.h5"), "r")
db = f["objects"]
names = list(db.keys())

for name in names:
    try:
        print("Reading", name)    
        binary = db[name]["object_model"][0]
        mesh = trimesh.load(trimesh.util.wrap_as_stream(binary), "ply")
        mesh.export(CWD / "kappler" / (name + ".obj"))
    except ValueError as e:
        print("Error, skipping", name)
        continue

## Train/Test Split

In [ ]:
train_urdfs =  [path for path in Path("train").iterdir() if path.suffix == ".urdf"]
test_urdfs = np.random.choice(urdfs, size=40, replace=False)

In [ ]:
for path in test_urdfs: 
    urdf_name = path.name
    visual_name = path.stem + "_visual.obj"
    collision_name = path.stem + "_collision.obj"
    
    shutil.move(str(Path("train") / urdf_name), str(Path("test") / urdf_name))
    shutil.move(str(Path("train") / visual_name), str(Path("test") / visual_name))
    shutil.move(str(Path("train") / collision_name), str(Path("test") / collision_name))